In [ ]:
!pip install tensorflow
!pip install --upgrade scikeras

In [ ]:
import numpy as np

import tensorflow as tf

from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, models

from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score

from scikeras.wrappers import KerasClassifier


def buildModel(learning_rate, num_filters):
  model = models.Sequential()
  #Define filters and convolutional layers here
  model.add(layers.Conv2D(num_filters, kernel_size=(3, 3), activation='relu',
  input_shape=(28, 28, 1)))
  #Add a maxpooling layer
  model.add(layers.MaxPooling2D(pool_size=(2, 2)))
  #Flatten the output and give it to a fully connected layer
  model.add(layers.Flatten())
  #One hidden layer maps the flattened neurons to output
  model.add(layers.Dense(10, activation="softmax"))
  model.compile(
      optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
      loss='categorical_crossentropy',
      metrics=['accuracy']
  )

  return model

def exploreParams(kf: StratifiedKFold, x_train, y_train):
  numFilters = [16, 32]
  learningRates = [0.001, 0.01]

  bestScore = 0
  bestParams = {
      "numFilters": numFilters[0],
      "learningRate": learningRates[0]
  }

  for filter in numFilters:
    for learningRate in learningRates:
      f1Scores = []

      for train_index, test_index in kf.split(x_train, y_train):
        x_train_fold, x_test_fold = x_train[train_index], x_train[test_index]
        y_train_fold, y_test_fold = y_train[train_index], y_train[test_index]

        y_train_cat = to_categorical(y_train_fold)

        pipeline = Pipeline([
            ("ConvolutionalModel", KerasClassifier(
                model = buildModel,
                num_filters = filter,
                learning_rate = learningRate,
                epochs=10,
                batch_size=32
            ))
        ])

        # Fit pipeline to training fold
        pipeline.fit(x_train_fold, y_train_cat)

        # Get predicted value off model
        y_pred = pipeline.predict(x_test_fold)
        y_pred_class = np.argmax(y_pred, axis=1)

        f1Scores.append(f1_score(y_test_fold, y_pred_class, average="weighted"))

    avgF1 = np.average(f1Scores)

    if avgF1 > bestScore:
      bestScore = avgF1
      bestParams['numFilters'] = filter
      bestParams['learningRate'] = learningRate

  return bestParams

if __name__ == "__main__":
  # Load data
  (x_train, y_train), (x_test, y_test) = mnist.load_data()

  # One Hot encode data
  y_train_encoded = to_categorical(y_train)


  kf = StratifiedKFold(n_splits=5, shuffle=True)

  bestParams = exploreParams(kf, x_train, y_train)

  pipeline = Pipeline([
      ("ConvolutionalModel", KerasClassifier(
          model = buildModel,
          num_filters = bestParams["numFilters"],
          learning_rate = bestParams["learningRate"],
          epochs=10,
          batch_size=32
      ))
  ])

  pipeline.fit(x_train, y_train_encoded)
  y_pred = pipeline.predict(x_test)
  y_pred_class = np.argmax(y_pred, axis=1)

  finalF1Score = f1_score(y_test, y_pred_class, average="weighted")

  print(f"The final f1 score is '{finalF1Score}' given by the parameters {bestParams}")
















-------------
[[[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 ...

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]]
-------------
[5 0 4 ... 5 6 8]
-------------
Epoch 1/10
1500/1500 [==============================] - 6s 3ms/step - loss: 0.6351 - accuracy: 0.9289
Epoch 2/10
1500/1500 [==============================] - 4s 3ms/s